In [1]:
import pandas as pd
import numpy as np

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [4]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1839
3,tt0000004,5.5,178
4,tt0000005,6.2,2624


# Preprocessing |  AKAS

In [7]:
akas = akas[akas['region'] == 'US']

In [8]:
#Replace "\N" with np.nan
akas.replace({'\\N': np.nan}, inplace=True)

# Preprocessing | TITLE BASICS

In [9]:
#Replace "\N" with np.nan
basics.replace({'\\N': np.nan}, inplace=True)
basics.head(15)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,NaN,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,NaN,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,NaN,1,"Documentary,Short"


In [10]:
#Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [11]:
#Eliminate movies that are null for genre
basics.dropna(subset=['genres'], inplace=True)

In [12]:
#keep only titleType==Movie
basics = basics[basics['titleType'] == 'Movie']

In [13]:
#keep startYear 2000-2022
basics = basics [(basics ['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

In [14]:
#Eliminate movies that include "Documentary" in genre (see tip below)
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics  =basics [~is_documentary]

In [15]:
#Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

Series([], Name: tconst, dtype: bool)

In [16]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


# Preprocessing |  RATINGS

In [17]:
#Replace "\N" with np.nan
ratings.replace({'\\N': np.nan}, inplace=True)

In [18]:
# Filter the ratings table down to only include the US by using the filter akas dataframe
keepers =ratings['tconst'].isin(akas['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1325151    False
1325152    False
1325153    False
1325154    False
1325155    False
Name: tconst, Length: 1325156, dtype: bool

In [19]:
ratings = ratings[keepers]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2624
5,tt0000006,5.1,182
6,tt0000007,5.4,821
...,...,...,...
1325117,tt9916200,8.1,230
1325118,tt9916204,8.2,264
1325125,tt9916348,8.3,18
1325126,tt9916362,6.4,5391


# Check files

In [20]:
akas.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448208 entries, 5 to 36400744
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1448208 non-null  object
 1   ordering         1448208 non-null  int64 
 2   title            1448208 non-null  object
 3   region           1448208 non-null  object
 4   language         3975 non-null     object
 5   types            980689 non-null   object
 6   attributes       46873 non-null    object
 7   isOriginalTitle  1446866 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.4+ MB


In [21]:
basics.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          0 non-null      object
 1   titleType       0 non-null      object
 2   primaryTitle    0 non-null      object
 3   originalTitle   0 non-null      object
 4   isAdult         0 non-null      object
 5   startYear       0 non-null      object
 6   endYear         0 non-null      object
 7   runtimeMinutes  0 non-null      object
 8   genres          0 non-null      object
dtypes: object(9)
memory usage: 0.0+ bytes


In [22]:
ratings.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502397 entries, 0 to 1325131
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         502397 non-null  object 
 1   averageRating  502397 non-null  float64
 2   numVotes       502397 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


# Saving Compressed csv.gz Files

In [23]:
# example making new folder with os
import os
os.makedirs('DataProject3/',exist_ok=True) 
# Confirm folder created
os.listdir("DataProject3/")

['title_basics.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'Project 3 IMDB.ipynb',
 'title_ratings.csv.gz']

In [24]:
## Save current dataframe to file.
basics.to_csv("DataProject3/title_basics.csv.gz",compression='gzip',index=False)

In [25]:
# Open saved file and preview again
basics = pd.read_csv("DataProject3/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [26]:
## Save current dataframe to file.
akas.to_csv("DataProject3/title_akas.csv.gz",compression='gzip',index=False)

In [27]:
# Open saved file and preview again
akas = pd.read_csv("DataProject3/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [28]:
## Save current dataframe to file.
ratings.to_csv("DataProject3/title_ratings.csv.gz",compression='gzip',index=False)

In [29]:
# Open saved file and preview again
ratings = pd.read_csv("DataProject3/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000005,6.2,2624
3,tt0000006,5.1,182
4,tt0000007,5.4,821
